<h3>Парсинг объявлений о продаже автомобилей с Авито</h3>

In [1]:
# импорт библиотек
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException 
from bs4 import BeautifulSoup

Будем брать популярные марки машин (те, у которых больше 10 объявлений)

In [2]:
# Количество объявлений каждой модели на 06.08.2024
dict_of_num_cars_of_models = {
    "Acura": 3,
    "AITO": 2,
    "Alfa Romeo": 2,
    "Alpina": 1,
    "Audi": 275,
    "Avatr": 2,
    "BAIC": 9,
    "BAW": 4,
    "Belgee": 3,
    "Bentley": 2,
    "BMW": 510,
    "Brilliance": 3,
    "BYD": 12,
    "Cadillac": 11,
    "Changan": 45,
    "Chery": 199,
    "CheryExeed": 3,
    "Chevrolet": 796,
    "Chrysler": 10,
    "Citroen": 129,
    "Dacia": 1,
    "Dadi": 1,
    "Daewoo": 144,
    "Daihatsu": 9,
    "Datsun": 134,
    "DKW": 1,
    "Dodge": 28,
    "Dongfeng": 8,
    "DW Hower": 5,
    "EXEED": 46,
    "FAW": 18,
    "FIAT": 54,
    "Ford": 484,
    "Forthing": 2,
    "Foton": 1,
    "GAC": 4,
    "Geely": 185,
    "Genesis": 8,
    "GMC": 1,
    "Great Wall": 76,
    "Hafei": 3,
    "Haima": 3,
    "Haval": 80,
    "Honda": 126,
    "Hongqi": 2,
    "Hummer": 9,
    "Hyundai": 962,
    "Infiniti": 61,
    "Iran Khodro": 6,
    "Isuzu": 3,
    "Iveco": 4,
    "JAC": 9,
    "Jaguar": 15,
    "Jeep": 43,
    "Jetour": 10,
    "JMC": 1,
    "Kaiyi": 15,
    "Kia": 194,
    "Lamborghini": 2,
    "Land Rover": 116,
    "LDV": 1,
    "Lexus": 146,
    "LIFAN": 130,
    "Lincoln": 3,
    "LiXiang": 20,
    "Mazda": 311,
    "Mercedes-Benz": 373,
    "MINI": 9,
    "Mitsubishi": 390,
    "Nissan": 611,
    "OMODA": 12,
    "Opel": 312,
    "Ora": 1,
    "Packard": 1,
    "Peugeot": 184,
    "Polar Stone (Jishi)": 1,
    "Pontiac": 2,
    "Porsche": 51,
    "RAM": 7,
    "Ravon": 7,
    "Renault": 649,
    "Rolls-Royce": 1,
    "Rover": 4,
    "Saab": 3,
    "SEAT": 4,
    "Shuanghuan": 1,
    "Skoda": 358,
    "Solaris": 3,
    "Soueast": 2,
    "SsangYong": 75,
    "Subaru": 120,
    "Suzuki": 88,
    "Tank": 30,
    "Tatra": 1,
    "Tesla": 7,
    "Toyota": 1007,
    "Volkswagen": 698,
    "Volvo": 59,
    "Vortex": 8,
    "Voyah": 22,
    "Wanderer": 1,
    "Wey": 2,
    "Willys": 1,
    "Xiaomi": 1,
    "Zeekr": 19,
    "ZOTYE": 4,
    "Автокам": 1,
    "Богдан": 4,
    "ВАЗ (LADA)": 5593,
    "ВИС": 73,
    "ГАЗ": 597,
    "ЗАЗ": 36,
    "ЗИЛ": 2,
    "ИЖ": 14,
    "ЛуАЗ": 10,
    "Москвич": 17,
    "РАФ": 3,
    "ТагАЗ": 5,
    "УАЗ": 212
}

In [ ]:
# Оставим только те модели, у которых количество объявлений > 10
car_marks = []

for mark in dict_of_num_cars_of_models.keys():
    if dict_of_num_cars_of_models[mark] > 10:
        car_marks.append(mark)

In [4]:
# начальная страница парсинга
url = 'https://www.avito.ru/samarskaya_oblast/avtomobili/s_probegom-ASgBAgICAUSGFMjmAQ'

Напишем функцию для решения Captcha

In [5]:
def kill_captcha(driver):
    wait = WebDriverWait(driver, timeout=1)
    try:
        # Проверка на наличие капчи
        captcha_element = wait.until(EC.presence_of_element_located(
                    (By.XPATH, "//h2[@class='firewall-title']")), message="Капча обнаружена")

        if captcha_element:
            print("Капча обнаружена. Решите ее вручную.")
            input("Нажмите Enter, когда решите капчу...")

    except TimeoutException:
        pass

Напишем функцию, которая будет закрывать страницу, если объявление недоступно

In [6]:
def is_unavailable_ad(driver):
    wait = WebDriverWait(driver, timeout=1)
    try:
        # Проверка на недоступное объявление
        blocked_ad = wait.until(EC.presence_of_element_located(
                    (By.XPATH, "//span[text()='Перейти к поиску']")))

        if blocked_ad:
            print("Недоступное объявление!")
            driver.find_element(
                By.XPATH, "//span[text()='Перейти к поиску']").click()
            return True

    except TimeoutException:
        return False

Напишем функцию получения данных со страницы объявления 

In [7]:
def get_car_data(driver):
    wait = WebDriverWait(driver, 1)  

    # Проверка на капчу
    kill_captcha(driver)

    # Проверка на недоступное объявление 
    if is_unavailable_ad(driver):
        return

    # Ожидание загрузки элементов с информацией о машине
    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//span[@itemprop='name']"))) 
    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//li[@class='params-paramsList__item-_2Y2O']"))) 
    wait.until(EC.presence_of_element_located((By.XPATH, "//span[@itemprop='price']")))

    car_data = {}

    # Парсинг объявления
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    base_info = [x.text.strip() for x in soup.find_all('span', itemprop='name')]
    mark, model = base_info[-3], base_info[-2]

    if mark not in car_marks:
        return

    specs = [x.text.strip() for x in soup.find_all('li', class_='params-paramsList__item-_2Y2O')]
    car_data = {item.split(': ')[0]: item.split(': ')[1] for item in specs}
    car_data['Название'] = mark + " " + model

    # Получение цены с использованием явного ожидания
    price_element = wait.until(EC.presence_of_element_located((By.XPATH, "//span[@itemprop='price']")))
    car_data['Цена'] = price_element.get_attribute('content')

    return car_data

Напишем фукнцию парсинга всех объявлений

In [8]:
def parse_cars(url, count_pages=1):
    driver = webdriver.Chrome()

    driver.get(url)

    # Ожидание загрузки страницы
    wait = WebDriverWait(driver, 5)

    # Проверка на капчу
    kill_captcha(driver)

    wait.until(EC.presence_of_element_located(
        (By.XPATH, "//div[span[@class='styles-module-switcherCircle-NWY_O']]")))

    # Включение фильтра "Сначала из Самарской области"
    driver.find_element(
        By.XPATH, "//div[span[@class='styles-module-switcherCircle-NWY_O']]").click()

    # Ожидание загрузки кнопки "Показать ... объявлений"
    wait.until(EC.presence_of_element_located(
        (By.XPATH, "//button[@data-marker='search-filters/submit-button']")))

    # Клик по кнопке "Показать ... объявлений"
    driver.find_element(
        By.XPATH, "//button[@data-marker='search-filters/submit-button']").click()

    cars = []

    # Парсинг всех страниц с объявлениями
    for cur_page in range(count_pages):
        # Ожидание загрузки ссылок на объявления
        wait.until(EC.presence_of_all_elements_located(
            (By.XPATH, "//a[@itemprop='url' and @data-marker='item-title']")))

        # Сбор ссылок на объявления
        car_links = [x.get_attribute("href") for x in driver.find_elements(
            By.XPATH, "//a[@itemprop='url' and @data-marker='item-title']")]

        # Парсинг каждого объявления на странице
        for car_link in car_links:
            # Открытие страницы объявления
            driver.execute_script(f"window.open('{car_link}');")

            # Переключение на новое окно
            driver.switch_to.window(driver.window_handles[-1])

            # Проверка на капчу
            kill_captcha(driver)

            # Получение данных со страницы объявления
            car_data = get_car_data(driver)

            # Добавление данных в список объявлений
            if car_data:
                cars.append(car_data)

            # Закрытие окна объявления
            driver.close()

            # Переключение на исходное окно
            driver.switch_to.window(driver.window_handles[0])

        if cur_page != count_pages - 1:
            # Ожидание загрузки кнопки "Следующая страница"
            wait.until(EC.presence_of_element_located(
                (By.XPATH, "//a[@aria-label='Следующая страница']")))
            driver.find_element(
                By.XPATH, "//a[@aria-label='Следующая страница']").click()

    driver.quit()

    return cars

In [9]:
# Парсинг
cars = parse_cars(url, 100)

Капча обнаружена. Решите ее вручную.
Недоступное объявление!
Недоступное объявление!


Преобразуем собранные данные в **DataFrame** и сохраним в файл

In [10]:
import pandas as pd

cars = pd.DataFrame(cars)

cars.to_csv('data/car_price.csv')

In [15]:
cars.head()

,Год выпуска,Поколение,Пробег,История пробега,ПТС,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Название,Цена,Обмен
0,2010,I рестайлинг (2009—2012),156 000 км,7 записей в отчёте Автотеки,Оригинал,2,Не битый,2.3 4WD AT (238 л.с.),2.3 л,Бензин,Автомат,Полный,Sport,Внедорожник 5-дверный,Белый,Левый,JMZE*************,Mazda CX-7,1050000,NaN
1,2020,G07 (2018—2022),109 000 км,Проверить в Автотеке,Оригинал,1,Не битый,M50d 3.0 xDrive Steptronic (400 л.с.),3 л,Дизель,Автомат,Полный,Special by Individual,Внедорожник 5-дверный,Синий,Левый,WBAC*************,BMW X7,8800000,Не интересует
2,2023,I (2023—2024),15 км,NaN,NaN,1,Не битый,2.0 T-GDI 4WD DCT (249 л.с.),2 л,Бензин,Робот,Полный,Platinum,Внедорожник 5-дверный,Чёрный,Левый,LVTD*************,EXEED RX,3730000,NaN
3,2024,I (2022—2024),300 км,2 записи в отчёте Автотеки,NaN,1,Не битый,1.5hyb 4WD AT (530 л.с.),1.5 л,Гибрид,Автомат,Полный,Executive,Седан,Чёрный,Левый,LDP9*************,Voyah Passion (Chasing Light),6770000,NaN
4,2024,I (2021—2024),10 км,NaN,Электронный,1,Не битый,2.0 4WD AT (238 л.с.),2 л,Бензин,Автомат,Полный,Flagship,Внедорожник 5-дверный,Белый,Левый,LB37*************,Geely Monjaro,3600000,Не интересует
